In [ ]:
import sys
sys.path.append('../src')
from OptimizationSolver import OptimizationSolver
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm

In [ ]:
% matplotlib notebook

# Set Initial Conditions

In [ ]:
InitialConditions = np.array([x for x in np.linspace(-1, 1, 100)])[:, np.newaxis]
TimeRange = [0, 5]
TimeOut = 501
ObjFunc = lambda x: x ** 3 + x ** 2 - 2 * x
DObjFunc = lambda x: 3 * x ** 2 + 2 * x - 2
DDObjFunc = lambda x: 6 * x + 2

# Run The Simulation

In [ ]:
optimization = OptimizationSolver(InitialConditions, TimeRange, TimeOut, ObjFunc, DObjFunc, DDObjFunc, Algorithm='Nesterov')
optimization.Simulate()

# Compute the Koopman Eigenfunctions

In [ ]:
fig, ax = plt.subplots()
plt.plot(optimization.Positions[:, 0, :].T);

In [ ]:
# Calculate the Koopman eigenfunctions for the Nesterov method
KoopmanIV = InitialConditions
KoopmanEigenfunction = np.zeros(optimization.Positions.shape, dtype=np.complex64)
KoopmanEigenfunction[:, :, 0] = InitialConditions
KoopmanEigenvalue = 1j
if KoopmanEigenvalue == 0:
    for i in range(KoopmanEigenfunction.shape[2] - 1):
        KoopmanEigenfunction[:, :, i + 1] = KoopmanEigenfunction[:, :, 0]
else:
    for i in range(KoopmanEigenfunction.shape[0]):
        for j in range(KoopmanEigenfunction.shape[2]):
            KoopmanEigenfunction[i, :, j] = KoopmanEigenfunction[i, :, 0] * np.exp(KoopmanEigenvalue * optimization.Times[j])

In [ ]:
colors = matplotlib.cm.ScalarMappable(cmap='jet')
color = colors.to_rgba(np.imag(KoopmanEigenfunction[:, 0, :]))
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111, projection='3d')
for i in range(optimization.Positions.shape[0]):
    ax.scatter(optimization.Positions[i, :, :], optimization.Times[:], optimization.Velocities[i, :, :], c=color[i, :, :]);
ax.set_xlabel('Position'); ax.set_ylabel('Time'); ax.set_zlabel('Velocity');
ax.set_title(f'Nesterov Colored by Imaginary Koopman for Lambda = {KoopmanEigenvalue}');
ax.view_init(35, 40)

In [ ]:
plt.savefig(fname='../data/Koopman/Nesterov Koopman(Imag) Cubic Lambda=1i.png', dpi='figure', bbox_inches='tight', pad_inches=0);